In [1]:
import cv2
import numpy as np
from arucoprocessor import ArucoProcessor
from tqdm import tqdm
import plotly.graph_objects as go

In [5]:
cap = cv2.VideoCapture('x_axis.mp4')

frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

buf = np.empty((frameCount, frameHeight, frameWidth, 3), np.dtype('uint8'))

fc = 0
ret = True

while (fc < frameCount and ret):
    ret, buf[fc] = cap.read()
    fc += 1

cap.release()

In [6]:
positions = []
apc = ArucoProcessor()

for i in tqdm(range(len(buf))):
    if apc.process_frame(buf[i]):
        pose = apc.get_pose_wrt_aruco()
        positions.append(pose[:3, 3])

positions = np.array(positions)

  6%|▌         | 4/66 [00:00<00:01, 36.42it/s]

Reliable


100%|██████████| 66/66 [00:01<00:00, 57.80it/s]


In [7]:
layout = go.Layout(
             scene=dict(
                 aspectmode='data'
         ))

fig = go.Figure(data=[go.Scatter3d(x=positions.T[0], y=positions.T[1], z=positions.T[2],
                                    marker_size=5, text=np.arange(0, len(positions)), line=dict(
        color='darkblue',
        width=2
    )
)], layout=layout)

fig.show()

In [5]:
fig.write_html("x_axis_interp.html")